In [ ]:
pip install folium

pip install haversine



In [127]:

import xml.etree.ElementTree as ET
import pprint
import random as rd
import folium as flMap
from haversine import haversine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import statistics



In [128]:


class UserObject:
    def __init__(self,id,lat_lon_obj,wsscr=None):
        self.id = id
        self.lat_lon_obj = lat_lon_obj
        self.wsscr=wsscr
        self.Positions = [lat_lon_obj]

    def addTrackPosition(self,pos):
        self.Positions.append(pos)

    def getTrackPosition(self):
        return self.Positions


class LatLon:
    def __init__(self,lat,lon):
        self.lat = float(lat)
        self.lon = float(lon)


class SurveillanceCode:
    WD = ["SUN","MON","TUE","WED","THU","FRI","SAT"]
    TS = [("T1","T3"),("T4","T6"),("T7","T9"),("T10","T12"),("T13","T15"),("T16","T18"),("T19","T21"),("T22","T24")]
    TM = ["flying","staying","vehicle","walking"]


    def __init__(self,userid,tm,wd,ts,lat_lon):
        self.userid = userid
        self.tm = tm
        self.wd = wd
        self.ts = ts
        self.lat_lon = lat_lon

        self.GeneratePositionIdentifier()
        

    def GeneratePositionIdentifier(self):
        entropy = rd.randint(100,999)
        self.id = self.userid.split()[0] + self.tm[:2] + self.wd[:2] + self.ts[0] + str(entropy)


class WSSCR:
    k_anonimity_threshold = 5
    cloaked_search_space_radius= 1  # kilo meters
    def __init__(self,WSSCR_id,WSSCR_list):
        self.id = WSSCR_id
        self.wsscr_users = WSSCR_list      # userlists
    
    def center_of_wsscr(self):
        self.wsscr_center = None
        if(len(self.wsscr_users)>0):
            lat_mean = statistics.mean([ user.lat_lon_obj.lat for user in self.wsscr_users])
            lon_mean = statistics.mean([ user.lat_lon_obj.lon for user in self.wsscr_users])

            return (lat_mean,lon_mean)
            



In [129]:
# parsing xml file for lat lng

tree = ET.parse('route_2.gpx')
root = tree.getroot()

#print(root.tag)

lat_lon_list=[]
for i in root.iter("wpt"):
    lat_lon_list.append(LatLon(i.attrib['lat'],i.attrib['lon']))


#pprint.PrettyPrinter().pprint(lat_lon_list)

In [130]:
    

def userIdGenerator():
    id=rd.randint(100,1000)
    return "user" + str(id)

def addWSSCR(user,userList):
    t_diff=0
    radius = WSSCR.cloaked_search_space_radius
    min = float('inf')
    for i in userList:
        diff = round(haversine((user.lat_lon_obj.lat,user.lat_lon_obj.lon),(i.lat_lon_obj.lat,i.lat_lon_obj.lon)),3)
        t_diff=diff
        #print(diff)
        if(diff<radius):
            if(diff<min):
                if(i.wsscr):
                    user.wsscr = i.wsscr
    if(user.wsscr==None):
        user.wsscr = "WSSCR-Region:"+ str(rd.randint(100,1000))
    
    #print(t_diff,user.wsscr)


userList = []
temp_lat_lon_list = lat_lon_list.copy()
for i in lat_lon_list:
    userLatLon =  rd.choice(temp_lat_lon_list)
    user = UserObject(userIdGenerator(),userLatLon)
    userList.append(user)    
    temp_lat_lon_list.remove(i)

for i in userList:
    addWSSCR(i,userList)




In [131]:


wsscr_list=[]

for i in userList:
    if(i.wsscr in wsscr_list):
        continue
    wsscr_list.append(i.wsscr)

wsscr_dict = {}

for i in wsscr_list:
    #count = 0
    wsscr_dict[i]=[]
    for j in userList:
        if(j.wsscr == i):
            #count += 1 
            wsscr_dict[i].append(j)

# pprint.PrettyPrinter().pprint(wsscr_dict)

wsscr_objects_list =[]

for x in wsscr_dict:
    wsscr= WSSCR(x,wsscr_dict[x])
    wsscr_objects_list.append(wsscr)


#pprint.PrettyPrinter().pprint(wsscr_objects_list)
# pprint.PrettyPrinter().pprint(wsscr_objects_list[1].center_of_wsscr())
# map

    

In [133]:
map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=14, tiles="Stamen Terrain")

# for i in userList:
#     flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.wsscr}, {i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)


for wsscr in wsscr_objects_list:
    if(len(wsscr.wsscr_users)>0):
        for user in wsscr.wsscr_users:
                flMap.Marker([user.lat_lon_obj.lat,user.lat_lon_obj.lon],popup=f"{user.wsscr}, \n{user.lat_lon_obj.lat},{user.lat_lon_obj.lon}",tooltip=f"<i>{user.id}</i>").add_to(map)
    flMap.Circle(
        location=list(wsscr.center_of_wsscr()),
        radius=1000*WSSCR.cloaked_search_space_radius + 500,
        popup=wsscr.id,
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
        ).add_to(map)



u1 = UserObject("testing User",LatLon(30.87374,75.81251))


#showing map
map



In [134]:
# function to generate position track
# this can be done by real time GPS Extraction of all neighbor users

def PathGeneration(user=None,val=0.0012):
    if(user):
        entropy = val

        lat = user.lat_lon_obj.lat + entropy
        lon = user.lat_lon_obj.lon + entropy

        user.addTrackPosition(LatLon(lat,lon))

        entropy +=  entropy
    else:
        for user in userList:
            entropy = val

            lat = user.lat_lon_obj.lat + entropy
            lon = user.lat_lon_obj.lon + entropy

            user.addTrackPosition(LatLon(lat,lon))

            # flMap.Marker([lat,lon],
            #             color=color,
            #             icon=flMap.Icon(color=color, icon='info-sign'),
            #             popup=f"{user.wsscr}, {lat},{lon}",
            #             tooltip=f"<i>{user.id}</i>").add_to(map)

            entropy +=  entropy


# just plotting all tracks on map

def VerificationOfTrack():
     for user in userList:
        user_tracks =user.getTrackPosition()
        color_list = ["red","blue","green","black"]
        if(len(user_tracks)>0):
            count = 0        
            for latlon in user_tracks:        
                color = color_list[count%len(color_list)]
                lat = latlon.lat 
                lon = latlon.lon
                flMap.Marker([lat,lon],
                            color=color,
                            icon=flMap.Icon(color=color, icon='info-sign'),
                            popup=f"{user.wsscr}, {lat},{lon}",
                            tooltip=f"<i>{user.id}</i>").add_to(map)
                count += 1


#path generation of all users near to MEC

PathGeneration(val = 0.0022)
PathGeneration(val = 0.0033)
PathGeneration(val = 0.0044)
PathGeneration(val = 0.00567)
PathGeneration(val = 0.00667)
#VerificationOfTrack()

#path generation of PBS user 
# prev history path of PBS user

PathGeneration(u1,val = 0.0022)
PathGeneration(u1,val = 0.0033)
PathGeneration(u1,val = 0.0044)
PathGeneration(u1,val = 0.00567)
PathGeneration(u1,val = 0.00667)




#map



In [135]:

#implementin knn algorithm
  
# Loading data
temp_list = [[user.id,user.lat_lon_obj.lat,user.lat_lon_obj.lon,user.wsscr] for user in userList]
dataset = pd.DataFrame(temp_list,columns=["User ID","Lat","Lon","WSSCR ID"])

#print(dataset)

# # Create feature and target arrays
x = dataset.iloc[:,1:3].values
y = dataset.iloc[:,-1].values
  
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)


knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=10)

In [ ]:
# print(knn.predict(X_test))
#y_pred = knn.predict(X_test)
#print(y_pred)

#print(classification_report(y_test,y_pred))
#print(confusion_matrix(y_test,y_pred))


In [136]:
data =[[u1.lat_lon_obj.lat,u1.lat_lon_obj.lon]]

predicted_wsscr = knn.predict(data)
u1.wsscr= predicted_wsscr
print(u1.wsscr)

print(wsscr_list)
flMap.Marker([u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],
                color='green',
                icon=flMap.Icon(color='green', icon='info-sign'),
                popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",
                tooltip=f"<i>{u1.id}</i>").add_to(map)


map

['WSSCR-Region:979']
['WSSCR-Region:248', 'WSSCR-Region:802', 'WSSCR-Region:979', 'WSSCR-Region:151', 'WSSCR-Region:520', 'WSSCR-Region:675', 'WSSCR-Region:868']


In [137]:
### Result of Position Recognition phase is : set of position of all users of the resulted WSSCR 

# output:> u1 (PBS user object ) ( having id, positions, WSSCR)

# subsequence code generation of PBS user

'''
assumptions are:

user's TM: walking
user's WD: SUN

'''
def SurveillanceCodeSequenceGeneration(user):
    temp=[]
    ind = 0
    for pos in user.getTrackPosition():
        ind += 1
        temp.append(SurveillanceCode(user.id,SurveillanceCode.TM[3],SurveillanceCode.WD[0],SurveillanceCode.TS[ind],pos))
    return temp

SurveillanceCode_list = SurveillanceCodeSequenceGeneration(u1)

#print(SurveillanceCode_list)
#experimental track seqeuence generation
def ExperimentalTrackSequenceGeneration(sur_list):
    TrackSequence = sur_list.copy()
    TSL =[]
    for i in range(10):
        randomness = rd.randint(1,10)
        #print(randomness)
        TrackSequence = sur_list.copy()
        for i in range(randomness):
            rd.shuffle(TrackSequence)
        TSL.append(TrackSequence)

    return TSL


TrackSequenceList = ExperimentalTrackSequenceGeneration(SurveillanceCode_list)

# print(TrackSequenceList)
# pprint.PrettyPrinter().pprint(TrackSequenceList)


# for i in SurveillanceCode_list:
#     print(i.tm)


In [138]:
#HMM Position Prediction of PBS user on the basis of Survellance code

def make_markov_model(sur_list, n_gram=2):
    markov_model = {}
    for i in range(len(sur_list)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state = sur_list[i+j]
            next_state = sur_list[i+j+n_gram]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

t = make_markov_model(SurveillanceCode_list)
print(t)

{<__main__.SurveillanceCode object at 0x000002748C0E7490>: {<__main__.SurveillanceCode object at 0x000002748C0E74C0>: 1.0}, <__main__.SurveillanceCode object at 0x000002748C0E7D60>: {<__main__.SurveillanceCode object at 0x000002748C0E74F0>: 1.0}, <__main__.SurveillanceCode object at 0x000002748C0E74C0>: {<__main__.SurveillanceCode object at 0x000002748C0E4280>: 1.0}}
